### Installing and Importing important libraries

In [ ]:
%pip install beautifulsoup4 --q
%pip install requests --q
%pip install undetected-chromedriver --q
%pip install selenium --q
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
import time
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
import pandas as pd

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Getting list of R1 Univeristies from Wikipedia using beautiful soup

In [ ]:


def extract_table_from_wikipedia(url):
    # Fetch the webpage content
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for HTTP errors

    # Parse the content with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the first table on the page
    table = soup.find('table')

    if not table:
        return None

    # Extract headers
    headers = [th.text.strip() for th in table.findAll("th")]

    # Extract rows
    rows = []
    for row in table.findAll("tr")[1:]:  # Skip the header row
        cells = row.findAll(["td", "th"])
        row_data = [cell.text.strip() for cell in cells]
        rows.append(row_data)

    # Convert to DataFrame
    df = pd.DataFrame(rows, columns=headers)

    return df

# Example usage:
url = "https://en.wikipedia.org/wiki/List_of_research_universities_in_the_United_States"  # Replace with the desired Wikipedia URL
df = extract_table_from_wikipedia(url)
df.head()

,Institution,Control,City,State
0,Arizona State University,Public,Tempe,AZ
1,Auburn University,Public,Auburn,AL
2,Baylor University,Private (non-profit),Waco,TX
3,Binghamton University,Public,Vestal,NY
4,Boston College,Private (non-profit),Chestnut Hill,MA


### Testing Undetected Chrome driver webscraping

In [ ]:
# Initialize undetected_chromedriver options
options = uc.ChromeOptions()
options.headless = False
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
options.add_argument(f'user-agent={user_agent}')


# Point the options to the correct chromedriver location
driver_path = r"C:\Users\Raymond\Desktop\Web Scraping\chromedriver-win64\chromedriver.exe"

# Create a new instance of the Chrome browser
browser = uc.Chrome(executable_path=driver_path, options=options)

# Navigate to a website
browser.get('https://openpayrolls.com/university-college/university-of-louisiana-at-lafayette/2022')
print(browser.title)


AttributeError: 'ChromeOptions' object has no attribute 'headless'

### Testing Scraping on the University of Louisiana at Lafayette

In [ ]:
# Initialize undetected_chromedriver options
options = uc.ChromeOptions()
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
options.add_argument(f'user-agent={user_agent}')

# Point the options to the correct chromedriver location
driver_path = r"C:\Users\Raymond\Desktop\Web Scraping\chromedriver-win64\chromedriver.exe"

# Create a new instance of the Chrome browser
browser = uc.Chrome(executable_path=driver_path, options=options)

data = []

# Loop through the pages
for page_num in range(3, 16):  # loop from 2 to 15
    try:
        page_url = f'https://openpayrolls.com/university-college/university-of-louisiana-at-lafayette/2022/page-{page_num}'

        browser.get(page_url)
        time.sleep(150)

        # Extract the main links
        main_link_elements = browser.find_elements(By.XPATH, '//table[@id="employeesTable"]//a[contains(@class, "btn-link font-weight-600") and .//span[@itemprop="name"]]')
        main_links = [link_element.get_attribute('href') for link_element in main_link_elements]

        # Navigate to each main link
        for main_link in main_links:
            try:
                browser.get(main_link)
                time.sleep(150)

                # Extract the secondary link within the table with the ID 'employmentRecordsTBody'
                sub_link_element = browser.find_element(By.XPATH, '//tbody[@id="employmentRecordsTBody"]//a[contains(@class, "text-primary text-nowrap font-weight-600")]')
                sub_link = sub_link_element.get_attribute('href')

                # Navigate to the secondary link
                browser.get(sub_link)
                time.sleep(150)

                # Extract details from the first table
                details1_elements = browser.find_elements(By.XPATH, '//div[contains(@class, "row mb-0 mb-sm-3")]//div[contains(@class, "row dt-table-row")]')
                details1 = {}
                for detail_element in details1_elements:
                    try:
                        key = detail_element.find_element(By.XPATH, './/div[1]').text
                        value = detail_element.find_element(By.XPATH, './/div[2]').text
                        details1[key] = value
                    except Exception as e:
                        print(f"Failed to extract detail from element: {e}")
                        continue

                # Extract details from the second table
                details2_elements = browser.find_elements(By.XPATH, '//table[contains(@class, "table table-sm FirstBold")]//tr')
                details2 = {}
                for detail_element in details2_elements:
                    try:
                        key = detail_element.find_element(By.XPATH, './/td[1]').text
                        value = detail_element.find_element(By.XPATH, './/td[2]').text
                        details2[key] = value
                    except Exception as e:
                        print(f"Failed to extract detail from element: {e}")
                        continue

                # Merge details and store in data list
                row_data = {**details1, **details2}
                data.append(row_data)

                # Update dataframe and print the latest row
                df3 = pd.DataFrame(data)
                print(df3.iloc[-1])

            except Exception as e:
                print(f"Failed to extract details from: {main_link}. Error: {e}")
                continue

    except Exception as e:
        print(f"Failed to process page: {page_num}. Error: {e}")
        continue

# Close the browser
browser.quit()

# Display the final dataframe
print(df3)


Name                                              Alicia B Moten
Location                                               Louisiana
PhoneAd                                        View phone number
Payroll Year                                                2022
Employer/Source             University of Louisiana At Lafayette
Employer/Location                     University of La-Lafayette
Job Title                             Administrative Assistant 3
Agency Code                                                  640
University/College    University of Louisiana At Lafayette (ULL)
Job Type                                               Full-Time
Payroll Type                                University & College
Pay Type                                              Classified
FTE                                                          100
Base/Regular Pay                                      $24,481.60
Name: 0, dtype: object
Name                                             Alicia F Guidry
Lo

Name                                           Allison H Cointot
Location                                               Louisiana
PhoneAd                                        View phone number
Payroll Year                                                2022
Employer/Source             University of Louisiana At Lafayette
Employer/Location                     University of La-Lafayette
Job Title                                             Instructor
Agency Code                                                  640
University/College    University of Louisiana At Lafayette (ULL)
Job Type                                               Full-Time
Payroll Type                                University & College
Pay Type                                            Unclassified
FTE                                                          100
Base/Regular Pay                                         $50,109
Name: 9, dtype: object
Name                                         Allison J Bourgeois
Lo

Failed to process page: 4. Error: Message: unknown error: net::ERR_INTERNET_DISCONNECTED
  (Session info: chrome=118.0.5993.90)
Stacktrace:
	GetHandleVerifier [0x008E4DE3+43907]
	(No symbol) [0x00870741]
	(No symbol) [0x007633ED]
	(No symbol) [0x0075FC4E]
	(No symbol) [0x007560B2]
	(No symbol) [0x00756F71]
	(No symbol) [0x00756395]
	(No symbol) [0x007556F8]
	(No symbol) [0x0075569A]
	(No symbol) [0x00754413]
	(No symbol) [0x00754AF8]
	(No symbol) [0x00765193]
	(No symbol) [0x007C5B61]
	(No symbol) [0x007B2B5C]
	(No symbol) [0x007C55CA]
	(No symbol) [0x007B2956]
	(No symbol) [0x0078E17E]
	(No symbol) [0x0078F32D]
	GetHandleVerifier [0x00B95AF9+2865305]
	GetHandleVerifier [0x00BDE78B+3163435]
	GetHandleVerifier [0x00BD8441+3138017]
	GetHandleVerifier [0x0096E0F0+605840]
	(No symbol) [0x0087A64C]
	(No symbol) [0x00876638]
	(No symbol) [0x0087675F]
	(No symbol) [0x00868DB7]
	BaseThreadInitThunk [0x7586FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77A17C6E+286]
	RtlGetAppContainerNamedObje

Failed to process page: 12. Error: Message: unknown error: net::ERR_INTERNET_DISCONNECTED
  (Session info: chrome=118.0.5993.90)
Stacktrace:
	GetHandleVerifier [0x008E4DE3+43907]
	(No symbol) [0x00870741]
	(No symbol) [0x007633ED]
	(No symbol) [0x0075FC4E]
	(No symbol) [0x007560B2]
	(No symbol) [0x00756F71]
	(No symbol) [0x00756395]
	(No symbol) [0x007556F8]
	(No symbol) [0x0075569A]
	(No symbol) [0x00754413]
	(No symbol) [0x00754AF8]
	(No symbol) [0x00765193]
	(No symbol) [0x007C5B61]
	(No symbol) [0x007B2B5C]
	(No symbol) [0x007C55CA]
	(No symbol) [0x007B2956]
	(No symbol) [0x0078E17E]
	(No symbol) [0x0078F32D]
	GetHandleVerifier [0x00B95AF9+2865305]
	GetHandleVerifier [0x00BDE78B+3163435]
	GetHandleVerifier [0x00BD8441+3138017]
	GetHandleVerifier [0x0096E0F0+605840]
	(No symbol) [0x0087A64C]
	(No symbol) [0x00876638]
	(No symbol) [0x0087675F]
	(No symbol) [0x00868DB7]
	BaseThreadInitThunk [0x7586FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77A17C6E+286]
	RtlGetAppContainerNamedObj

In [ ]:
df3.head(17)

,Name,Location,PhoneAd,Payroll Year,Employer/Source,Employer/Location,Job Title,Agency Code,University/College,Job Type,Payroll Type,Pay Type,FTE,Base/Regular Pay
0,Alicia B Moten,Louisiana,View phone number,2022,University of Louisiana At Lafayette,University of La-Lafayette,Administrative Assistant 3,640,University of Louisiana At Lafayette (ULL),Full-Time,University & College,Classified,100,"$24,481.60"
1,Alicia F Guidry,Louisiana,View phone number,2022,University of Louisiana At Lafayette,University of La-Lafayette,Supervisor,640,University of Louisiana At Lafayette (ULL),Full-Time,University & College,Unclassified,100,"$86,740"
2,Ali Ettehadi,Louisiana,View phone number,2022,University of Louisiana At Lafayette,University of La-Lafayette,Coordinator,640,University of Louisiana At Lafayette (ULL),Full-Time,University & College,Unclassified,100,"$40,000"
3,Alise C Hagan,Louisiana,View phone number,2022,University of Louisiana At Lafayette,University of La-Lafayette,Non-Teaching Professional,640,University of Louisiana At Lafayette (ULL),Full-Time,University & College,Unclassified,100,"$64,000"
4,Alisha S Bourque,Louisiana,View phone number,2022,University of Louisiana At Lafayette,University of La-Lafayette,Coordinator,640,University of Louisiana At Lafayette (ULL),Full-Time,University & College,Unclassified,100,"$63,345"
5,Alison A Deville,Louisiana,View phone number,2022,University of Louisiana At Lafayette,University of La-Lafayette,Coordinator,640,University of Louisiana At Lafayette (ULL),Full-Time,University & College,Unclassified,100,"$42,000"
6,Alison Pisani,Louisiana,View phone number,2022,University of Louisiana At Lafayette,University of La-Lafayette,Coordinator,640,University of Louisiana At Lafayette (ULL),Full-Time,University & College,Unclassified,100,"$36,951"
7,Allen L Bonnet,Louisiana,View phone number,2022,University of Louisiana At Lafayette,University of La-Lafayette,Director,640,University of Louisiana At Lafayette (ULL),Full-Time,University & College,Unclassified,100,"$71,791"
8,Allen M Latour,Louisiana,View phone number,2022,University of Louisiana At Lafayette,University of La-Lafayette,Technician,640,University of Louisiana At Lafayette (ULL),Full-Time,University & College,Unclassified,100,"$55,520"
9,Allison H Cointot,Louisiana,View phone number,2022,University of Louisiana At Lafayette,University of La-Lafayette,Instructor,640,University of Louisiana At Lafayette (ULL),Full-Time,University & College,Unclassified,100,"$50,109"


In [ ]:
df4.to_csv('df4.csv', index=False)

Scraping from multiple universities in Parallel

In [ ]:
import threading
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
import pandas as pd
import time

def scrape_page(url, file_name):
    options = uc.ChromeOptions()
    options.headless = False
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    options.add_argument(f'user-agent={user_agent}')

    # Point the options to the correct chromedriver location
    driver_path = r"C:\Users\Raymond\Desktop\Web Scraping\chromedriver-win64\chromedriver.exe"

    browser = None  # Initialize the browser variable outside the try block

    try:
        browser = uc.Chrome(executable_path=driver_path, options=options)
        data = []

        for page_num in range(1, 300):  # Adjust range as necessary
            try:
                page_url = f'{url}/page-{page_num}'
                browser.get(page_url)
                time.sleep(150)  # Adjust sleep time as needed

                main_link_elements = browser.find_elements(By.XPATH, '//table[@id="employeesTable"]//a[contains(@class, "btn-link font-weight-600") and .//span[@itemprop="name"]]')
                main_links = [element.get_attribute('href') for element in main_link_elements]

                for main_link in main_links:
                    retry_count = 0
                    while retry_count < 3:
                        try:
                            browser.get(main_link)
                            time.sleep(150)

                            sub_link_element = browser.find_element(By.XPATH, '//tbody[@id="employmentRecordsTBody"]//a[contains(@class, "text-primary text-nowrap font-weight-600")]')
                            sub_link = sub_link_element.get_attribute('href')
                            browser.get(sub_link)
                            time.sleep(150)

                            details1_elements = browser.find_elements(By.XPATH, '//div[contains(@class, "row mb-0 mb-sm-3")]//div[contains(@class, "row dt-table-row")]')
                            details1 = {}
                            for element in details1_elements:
                                key = element.find_element(By.XPATH, './/div[1]').text
                                value = element.find_element(By.XPATH, './/div[2]').text
                                details1[key] = value

                            details2_elements = browser.find_elements(By.XPATH, '//table[contains(@class, "table table-sm FirstBold")]//tr')
                            details2 = {}
                            for element in details2_elements:
                                key = element.find_element(By.XPATH, './/td[1]').text
                                value = element.find_element(By.XPATH, './/td[2]').text
                                details2[key] = value

                            row_data = {**details1, **details2}
                            data.append(row_data)
                            break

                        except Exception as e:
                            print(f"Attempt {retry_count + 1}: Failed to extract details from: {main_link}. Error: {e}")
                            retry_count += 1
                            if retry_count >= 3:
                                print(f"Skipping {main_link} after 3 failed attempts.")
                                break

            except Exception as e:
                print(f"Failed to process page {page_num} of {url}: {e}")
                continue

        df = pd.DataFrame(data)
        df.to_csv(file_name, index=False)

    except Exception as overall_error:
        print(f"An error occurred with the browser for {url}: {overall_error}")

    finally:
        if browser:
            browser.quit()

# URLs to scrape
urls = [
    # "https://openpayrolls.com/university-college/auburn-university/2021",
    # "https://openpayrolls.com/university-college/binghamton-university/2021",
    # "https://openpayrolls.com/university-college/clemson-university/2022",
#     "https://openpayrolls.com/university-college/colorado-school-of-mines/2022",
#     "https://openpayrolls.com/university-college/colorado-state-university/2022",
#     "https://openpayrolls.com/university-college/florida-international-university/2022",
#     "https://openpayrolls.com/university-college/florida-state-university/2022",
#     "https://openpayrolls.com/university-college/george-mason-university/2022",
#     "https://openpayrolls.com/university-college/georgia-institute-of-technology/2022",
#     "https://openpayrolls.com/university-college/georgia-state-university/2022",
#     "https://openpayrolls.com/university-college/the-graduate-center/2021",
#     "https://openpayrolls.com/university-college/indiana-university/2021",
#     "https://openpayrolls.com/university-college/iowa-state-university/2022",
#     "https://openpayrolls.com/university-college/kansas-state-university/2022",
#     "https://openpayrolls.com/university-college/louisiana-state-university-and-agricultural-mechanical-college/2022",
#     "https://openpayrolls.com/university-college/michigan-state-university/2022",
#     "https://openpayrolls.com/university-college/montana-state-university/2022",
#     "https://openpayrolls.com/university-college/new-jersey-institute-of-technology/2022",
#     "https://openpayrolls.com/university-college/north-carolina-state-university/2022",
#     "https://openpayrolls.com/university-college/north-dakota-state-university/2022",
#     "https://openpayrolls.com/university-college/oklahoma-state-university/2022",
#     "https://openpayrolls.com/university-college/old-dominion-university/2022",
#     "https://openpayrolls.com/university-college/pennsylvania-state-university/2021",
#     "https://openpayrolls.com/university-college/purdue-university/2021",
#     "https://openpayrolls.com/university-college/rutgers-university/2022",
#     "https://openpayrolls.com/university-college/stony-brook-university/2021",
#     "https://openpayrolls.com/university-college/temple-university/2021",
#     "https://openpayrolls.com/university-college/texas-am-agrilife-research/2022",
#     "https://openpayrolls.com/university-college/texas-am-agrilife-extension-service",
#     "https://openpayrolls.com/university-college/texas-tech-university/2022",
#     "https://openpayrolls.com/university-college/university-at-albany/2021",
#     "https://openpayrolls.com/university-college/university-at-buffalo/2021",
#     "https://openpayrolls.com/university-college/the-university-of-alabama/2021",
#     "https://openpayrolls.com/university-college/university-of-arizona/2023",
#     "https://openpayrolls.com/university-college/university-of-arkansas/2019",
#     "https://openpayrolls.com/university-college/university-of-california-berkeley/2021",
#     "https://openpayrolls.com/university-college/university-of-california-davis",
#     "https://openpayrolls.com/university-college/university-of-california-irvine/2021",
#     "https://openpayrolls.com/university-college/university-of-california-los-angeles/2021",
#     "https://openpayrolls.com/university-college/university-of-california-riverside/2021",
#     "https://openpayrolls.com/university-college/university-of-california-san-diego/2021",
#     "https://openpayrolls.com/university-college/university-of-california-santa-barbara/2021",
#     "https://openpayrolls.com/university-college/university-of-california-santa-cruz/2021",
#     "https://openpayrolls.com/university-college/university-of-central-florida/2022",
#     "https://openpayrolls.com/university-college/university-of-colorado-boulder/2022",
#     "https://openpayrolls.com/university-college/university-of-colorado-denver/2022",
#     "https://openpayrolls.com/university-college/university-of-connecticut/2022",
#     "https://openpayrolls.com/university-college/university-of-florida/2022",
#     "https://openpayrolls.com/university-college/university-of-georgia/2022",
#     "https://openpayrolls.com/university-college/university-of-hawaii-at-manoa/2016",
#     "https://openpayrolls.com/university-college/university-of-houston-university-park/2022",
#     "https://openpayrolls.com/university-college/university-of-illinois-at-chicago/2021",
#     "https://openpayrolls.com/university-college/university-of-illinois-at-urbana-champaign/2021",
#     "https://openpayrolls.com/university-college/university-of-iowa/2022",
#     "https://openpayrolls.com/university-college/university-of-kansas/2022",
#     "https://openpayrolls.com/university-college/university-of-kentucky/2021",
#     "https://openpayrolls.com/university-college/university-of-louisville/2021",
#     "https://openpayrolls.com/university-college/university-of-maine/2022",
#     "https://openpayrolls.com/university-college/university-of-maryland/2021",
#     "https://openpayrolls.com/university-college/university-of-massachusetts-amherst/2022",
#     "https://openpayrolls.com/university-college/university-of-memphis/2020",
#     "https://openpayrolls.com/university-college/university-of-michigan/2022",
#     "https://openpayrolls.com/university-college/university-of-minnesota-twin-cities/2022",
#     "https://openpayrolls.com/university-college/university-of-missouri-system/2022",
#     "https://openpayrolls.com/university-college/university-of-montana/2022",
#     "https://openpayrolls.com/university-college/university-of-nebraska/2022",
#     "https://openpayrolls.com/university-college/university-of-nevada-reno/2019",
#     "https://openpayrolls.com/university-college/university-of-new-hampshire/2022",
#     "https://openpayrolls.com/university-college/the-university-of-north-carolina-at-chapel-hill/2022",
#     "https://openpayrolls.com/university-college/university-of-north-texas/2022",
#     "https://openpayrolls.com/university-college/university-of-oklahoma/2022",
#     "https://openpayrolls.com/university-college/university-of-pittsburgh/2021",
#     "https://openpayrolls.com/university-college/university-of-south-carolina/2022",
#     "https://openpayrolls.com/university-college/university-of-south-florida/2022",
#     "https://openpayrolls.com/university-college/university-of-texas-at-arlington/2022",
#     "https://openpayrolls.com/university-college/university-of-texas-at-dallas/2022",
#     "https://openpayrolls.com/university-college/university-of-texas-at-el-paso/2022",
#     "https://openpayrolls.com/university-college/university-of-texas-at-san-antonio/2022",
#     "https://openpayrolls.com/university-college/university-of-utah/2021",
#     "https://openpayrolls.com/university-college/university-of-virginia/2022",
#     "https://openpayrolls.com/university-college/university-of-washington/2021",
#     "https://openpayrolls.com/university-college/university-of-wisconsin-madison/2022",
#     "https://openpayrolls.com/university-college/university-of-wisconsin-milwaukee/2022",
#     "https://openpayrolls.com/university-college/utah-state-university/2021",
#     "https://openpayrolls.com/university-college/virginia-polytechnic-institute-and-state-university-virginia-tech/2022",
#     "https://openpayrolls.com/university-college/washington-state-university/2021",
#     "https://openpayrolls.com/university-college/west-virginia-university/2022",
    # Add more URLs as needed
]

# Creating and starting threads
threads = []
for i, url in enumerate(urls):
    file_name = f'filess-{i+1}.csv'
    thread = threading.Thread(target=scrape_page, args=(url, file_name))
    thread.start()
    threads.append(thread)

# Waiting for all threads to finish
for thread in threads:
    thread.join()

print("Scraping complete.")


Scraping a set of R1 public universities per round in Parallel - due to connectivity issues arising from scrapping all 89 public Universities at once

In [ ]:

import threading
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import logging

# Initialize logging
logging.basicConfig(level=logging.INFO)

def scrape_page(url, file_name):
    options = uc.ChromeOptions()
    options.headless = False
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    options.add_argument(f'user-agent={user_agent}')
    driver_path = r"C:\Users\Raymond\Desktop\Web Scraping\chromedriver-win64\chromedriver.exe"
    browser = uc.Chrome(executable_path=driver_path, options=options)

    try:
        data = []
        for page_num in range(1, 300):  # Adjust range as necessary
            try:
                page_url = f'{url}/page-{page_num}'
                browser.get(page_url)
                WebDriverWait(browser, 180).until(EC.presence_of_element_located((By.XPATH, '//table[@id="employeesTable"]')))

                main_link_elements = browser.find_elements(By.XPATH, '//table[@id="employeesTable"]//a[contains(@class, "btn-link font-weight-600") and .//span[@itemprop="name"]]')
                main_links = [element.get_attribute('href') for element in main_link_elements]

                for main_link in main_links:
                    try:
                        browser.get(main_link)
                        WebDriverWait(browser, 180).until(EC.presence_of_element_located((By.XPATH, '//tbody[@id="employmentRecordsTBody"]')))

                        sub_link_element = browser.find_element(By.XPATH, '//tbody[@id="employmentRecordsTBody"]//a[contains(@class, "text-primary text-nowrap font-weight-600")]')
                        sub_link = sub_link_element.get_attribute('href')
                        browser.get(sub_link)
                        WebDriverWait(browser, 180).until(EC.presence_of_element_located((By.XPATH, '//div[contains(@class, "row mb-0 mb-sm-3")]')))

                        # Extracting details1
                        details1_elements = browser.find_elements(By.XPATH, '//div[contains(@class, "row mb-0 mb-sm-3")]//div[contains(@class, "row dt-table-row")]')
                        details1 = {}
                        for element in details1_elements:
                            key = element.find_element(By.XPATH, './/div[1]').text
                            value = element.find_element(By.XPATH, './/div[2]').text
                            details1[key] = value

                        # Extracting details2
                        details2_elements = browser.find_elements(By.XPATH, '//table[contains(@class, "table table-sm FirstBold")]//tr')
                        details2 = {}
                        for element in details2_elements:
                            key = element.find_element(By.XPATH, './/td[1]').text
                            value = element.find_element(By.XPATH, './/td[2]').text
                            details2[key] = value

                        row_data = {**details1, **details2}
                        data.append(row_data)

                    except Exception as e:
                        logging.warning(f"Error on URL: {main_link}, Page: {page_num}: {e}")

            except Exception as e:
                logging.error(f"Error on URL: {url}, Page: {page_num}: {e}")

        pd.DataFrame(data).to_csv(file_name, index=False)

    finally:
        if browser:
            browser.quit()

# URLs to scrape
urls = [
    "https://openpayrolls.com/university-college/university-of-louisville/2021-professor",
    "https://openpayrolls.com/university-college/university-of-maine/2022-professor",
    "https://openpayrolls.com/university-college/university-of-maryland/2021-professor",
    "https://openpayrolls.com/university-college/university-of-massachusetts-amherst/2022-professor",
    "https://openpayrolls.com/university-college/university-of-memphis/2020-professor",
    "https://openpayrolls.com/university-college/university-of-michigan/2022-professor",
    # List your URLs here
]

# Creating and starting threads
threads = []
for i, url in enumerate(urls):
    file_name = f'filie-{i+1}.csv'
    thread = threading.Thread(target=scrape_page, args=(url, file_name))
    thread.start()
    threads.append(thread)

# Waiting for all threads to finish
for thread in threads:
    thread.join()

print("Scraping complete.")


